<a href="https://colab.research.google.com/github/jeguns/EP7173/blob/main/Unidad%2005/05_Reducci%C3%B3n_de_dimensiones_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Preliminares**



In [ ]:
pip install factor_analyzer &> /dev/null

In [ ]:
pip install pca &> /dev/null

In [ ]:
import numpy as np
np.set_printoptions(precision=5, suppress=True) # suprime la notación científica, y muestra los resultados con 5 decimales

In [ ]:
import pandas as pd
pd.set_option('display.float_format', '{:.5f}'.format) # muestra los resultados en formato decimal con 5 decimales

En el conjunto de datos 05 - Datos - 01.xlsx se registran las puntuaciones obtenidas en 6 evaluaciones del curso Estadística 1.

La lista de atributos corresponde a puntuaciones obtenidas por estudiantes en evaluaciones cuyo valor máximo varia.

- P1: de 0 a 200

- P2: de 0 a 7

- P3: de 0 a 20

- P4: de 0 a 4

- P5: de 0 a 6

- P6: de 0 a 4



In [ ]:
datos1 = pd.read_excel('05 - Datos - 01.xlsx', na_values = ["-"])
datos1

In [ ]:
datos1 = datos1.drop(['ESTUDIANTE'], axis=1)

In [ ]:
datos1.info()

In [ ]:
datos1.head(10)

In [ ]:
datos1.describe()

Tomar atención con las variables P1 y P3 pues su promedio es  más alto que las demás.

Esto nos llevará a tomar la decisión de estandarizar los datos.

# **Análisis exploratorio**

## **Matriz de correlación**

In [ ]:
datos1.corr()

In [ ]:
datos1[['P2','P6']].corr()

In [ ]:
datos1[['P3','P4']].corr()

### **Gráficos de la matriz de correlación**

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(datos1.corr())

In [ ]:
import seaborn as sns
sns.heatmap(datos1.corr(),
            xticklabels=datos1.corr().columns.values,
            yticklabels=datos1.corr().columns.values)

In [ ]:
datos1.corr().style.background_gradient(cmap='coolwarm')

In [ ]:
datos1.corr().style.background_gradient(cmap='coolwarm').set_properties(**{'font-size': '0pt'})

### **Determinante de la matriz de correlación**

In [ ]:
np.linalg.det(datos1.corr()) # buscamos determinantes cercanos a 0.

In [ ]:
np.linalg.det(datos1[['P2','P6']].corr())

'P2' y 'P6' no están asociadas y su determinante es aproximadamente 1.

In [ ]:
np.linalg.det(datos1[['P3','P4']].corr())

'P2' y 'P3' están fuertemente asociadas y su determinante es más bajo.

## **KMO (Kaiser-Meyer-Olkin)**

KMO < 0.5 $\rightarrow$ no ejecutar el análisis factorial o de componentes principales

0.5 <= KMO < 0.7  $\rightarrow$ ok, ejecutar el análisis factorial o de componentes principales

0.7 <= KMO <= 1  $\rightarrow$ es el mejor escenario para aplicar el análisis factorial o de componentes principales

In [ ]:
from factor_analyzer.factor_analyzer import calculate_kmo
kmo_all, kmo_model = calculate_kmo(datos1)

In [ ]:
kmo_all

In [ ]:
kmo_model

## **Prueba de esfericidad de Bartlett**

$H_0: R = \mathbf{I}$

$H_1: R \neq \mathbf{I}$

$R:$ Matriz de correlaciones

$\mathbf{I}:$ Matriz identidad

Nostoros buscamos rechazar la hipótesis nula a fin de poder aplicar el análisis factorial o el análisis de componentes principales.

In [ ]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity
chi_cuadrado, pv = calculate_bartlett_sphericity(datos1)
chi_cuadrado, pv # mientras más grande el chi cuadrado, es más significativa la prueba

Dado que el pvalor es 0, se decide rechazar la hipótesis nula. Por lo tanto, sí existe asociación entre al menos un par de variables, y en consecuencia es factible aplicar el análisis factorial o el de componentes principales

# **Análisis de componentes principales**



In [ ]:
from pca import pca
import matplotlib.pyplot as plt

Usamos los datos originales (a pesar de que no es correcto, pero con la finalidad de ver la consecuencia de hacerlo)

In [ ]:
mod_cp  = pca(n_components = 6) # define acp
cp1     = mod_cp.fit_transform(datos1) # aplica acp
mod_cp.plot()

In [ ]:
cp1['variance_ratio']*100 # varianza explicada por eje

In [ ]:
cp1['explained_var']*100 # varianza explicada acumulada

In [ ]:
cp1['loadings'] # cargas

In [ ]:
cp1['PC'] # scores

In [ ]:
mod_cp.biplot(n_feat=13)

Es necesario estandarizar para evitar este problema

In [ ]:
from sklearn.preprocessing import StandardScaler
datos1_st = datos1.copy()
datos1_st[['P1','P2','P3','P4','P5','P6']] = StandardScaler().fit_transform(datos1_st[['P1','P2','P3','P4','P5','P6']]) # estandarizamos o escalamos (x-media)/sd

In [ ]:
mod_cp  = pca(n_components = 6) # define acp
cp2     = mod_cp.fit_transform(datos1_st) # aplica acp
mod_cp.plot()

In [ ]:
cp2['variance_ratio']*100 # varianza explicada por eje

In [ ]:
cp2['explained_var']*100 # varianza explicada acumulada

In [ ]:
cp2['loadings'] # cargas

In [ ]:
cp2['PC'] # scores o puntajes # objetivo final en el preprocesamiento

In [ ]:
cp2['PC'].info()

In [ ]:
cp2['PC'][['PC1','PC2']] # scores solo de los dos primeros ejes o componentes

In [ ]:
mod_cp.biplot(n_feat = 6)

In [ ]:
cp2['PC'].corr()

In [ ]:
plt.matshow(cp2['PC'].corr())

También es posible ejecutar el análisis de componentes principales usando los paquete sklearn y statmodels.

Por otro lado, en R, es posible ejecutar el análisis de componentes principales usando alguno de estos paquetes:

- stats::prcomp

- stats::princomp

- FactoMineR::PCA

- psych::principal

# **Análisis factorial**


In [ ]:
from factor_analyzer import FactorAnalyzer

In [ ]:
fa = FactorAnalyzer(rotation = None, n_factors = 6)
fa.fit(datos1_st)

In [ ]:
fa.get_factor_variance()

In [ ]:
pd.DataFrame(fa.get_factor_variance(), index=['Variance','Proportional Var','Cumulative Var']).T

In [ ]:
pd.DataFrame(fa.get_factor_variance(), index=['Variance','Proportional Var','Cumulative Var']).T['Proportional Var']

In [ ]:
sum(pd.DataFrame(fa.get_factor_variance(), index=['Variance','Proportional Var','Cumulative Var']).T['Proportional Var'])

In [ ]:
ev,_ = fa.get_eigenvalues() # LA cantidad de autovalores es la misma que la de variables
ev

In [ ]:
plt.scatter(range(1,datos1.shape[1]+1),ev)
plt.plot(range(1,datos1.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigen Value')
plt.grid()

Elegimos la cantidad de factores a ser retenidos:

- Según proporción de varianza explicada

- Según gráfico del codo

- Por conveniencia: 2

Considerando 3 factores:

In [ ]:
fa2 = FactorAnalyzer(rotation = None, n_factors = 3)
fa2.fit(datos1_st)

In [ ]:
pd.DataFrame(fa2.loadings_, index = datos1.columns) # loadings = cargas

In [ ]:
pd.DataFrame(fa2.get_communalities(), index = datos1.columns, columns = ['Comunalidades'])

In [ ]:
pd.DataFrame(fa2.get_uniquenesses(), index = datos1.columns, columns = ['Especificidades'])

In [ ]:
scores = pd.DataFrame(fa2.transform(datos1_st))
scores

In [ ]:
scores.corr()

In [ ]:
plt.matshow(scores.corr())

En R, es posible ejecutar el análisis factorial usando alguno de estos paquetes:

- stats::factanal

- psych::fa

# **Escalamiento Multidimensional**

### **Ejemplo 1**

El archivo 05 - Datos - 2.xlsx contiene las distancias, en km, entre 7 distintas ciudades de Perú

In [ ]:
datos2 = pd.read_excel('05 - Datos - 2.xlsx', index_col = 0)
datos2

In [ ]:
from sklearn.manifold import MDS
mds = MDS(dissimilarity = 'precomputed', n_components = 2, random_state = 10) # PASO 2
Ejes = mds.fit_transform(datos2) # PASO 3
Ejes

In [ ]:
Ejes = pd.DataFrame(Ejes,
                   index   = ['Arequipa','Cuzco','Chiclayo','Lima','Puno','Tacna','Iquitos'],
                   columns = ['Eje1','Eje2'] )
Ejes

In [ ]:
plt.scatter(Ejes[['Eje1']],Ejes[['Eje2']], marker= '*')
for i in range(Ejes.shape[0]):
 plt.text(x = Ejes.Eje1[i]+0.3,
          y = Ejes.Eje2[i]+0.3,
          s = Ejes.index[i])
plt.axvline(x=0.0)
plt.axhline(y=0.0)

In [ ]:
mds.stress_

In [ ]:
stress = []
for dim in range(2, 10):
    # Configurar el objeto MDS
    mds = MDS(dissimilarity = 'precomputed', n_components = dim, random_state = 10)
    # Aplicar MDS
    ejes = mds.fit_transform(datos2)
    # Obtener el estres
    stress.append(mds.stress_)
# Graficar stress vs. n_components
plt.plot(range(2, 10), stress)
plt.xticks(range(2, 10, 2))
plt.xlabel('n_components')
plt.ylabel('stress')
plt.show()

In [ ]:
stress

In [ ]:
mds2 = MDS(dissimilarity = 'precomputed', n_components = 3, random_state = 10)
Ejes = mds2.fit_transform(datos2)
Ejes = pd.DataFrame(Ejes,
                     index = ['Arequipa','Cuzco','Chiclayo','Lima','Puno','Tacna','Iquitos'],
                     columns = ['Eje1','Eje2','Eje3'] )
Ejes

In [ ]:
plt.scatter(Ejes[['Eje1']],Ejes[['Eje2']], marker= '.')
for i in range(Ejes.shape[0]):
 plt.text(x = Ejes.Eje1[i]+0.3,
          y = Ejes.Eje2[i]+0.3,
          s = Ejes.index[i])
plt.axvline(x=0.0)
plt.axhline(y=0.0)

In [ ]:
mds2.stress_

In [ ]:
pd.concat([datos2, Ejes], axis=1)

In [ ]:
pd.concat([datos2, Ejes], axis=1).corr()

### **Ejemplo 2**

El archivo 05 - Datos - 3.xlsx contiene puntajes referidos a programas de ciertos rubros(información, documentales, concursos, deportes, música) de 9 canales de televisión, a los que se les agrega el canal 10 (PERFECTO) y el canal 11 (EL PEOR DE TODOS).

In [ ]:
datos3 = pd.read_excel('05 - Datos - 3.xlsx', index_col = 0)
datos3

In [ ]:
# elegir la cantidad de ejes según el estress
stress = []
for dim in range(2, 10):
    # Configurar el objeto MDS
    mds = MDS(n_components = dim, random_state = 8) # dissimilarity, default='euclidean'
    # Aplicar MDS
    pts = mds.fit_transform(datos3)
    # Obtener el valor de estrés
    stress.append(mds.stress_)
# Graficar stress vs. n_components
plt.plot(range(2, 10), stress)
plt.xticks(range(2, 10, 2))
plt.xlabel('n_components')
plt.ylabel('stress')
plt.show()

In [ ]:
mds  = MDS(n_components = 4, random_state = 8)
Ejes = mds.fit_transform(datos3)
Ejes = pd.DataFrame(Ejes,
                    index = ['1','2','3','4','5','6','7','8','9','10','11'],
                    columns = ['Eje1','Eje2','Eje3','Eje4'] )
Ejes # scores o coordenadas

In [ ]:
plt.scatter(Ejes[['Eje1']],Ejes[['Eje2']], marker= '.')
for i in range(Ejes.shape[0]):
 plt.text(x = Ejes.Eje1[i]+0.3,
          y = Ejes.Eje2[i]+0.3,
          s = Ejes.index[i])
plt.axvline(x=0.0)
plt.axhline(y=0.0)

In [ ]:
m = Ejes[['Eje1','Eje2','Eje3']].to_numpy()
m

In [ ]:
from matplotlib import pyplot
from pylab import figure

fig = figure()
ax = fig.add_subplot(projection='3d')

for i in range(len(m)): #grafica cada punto + texto
    ax.scatter(m[i,0],m[i,1],m[i,2],color='b')
    ax.text(m[i,0],m[i,1],m[i,2],'%s' % (str(Ejes.index[i])), size = 15, color='k')

ax.set_xlabel('Eje 1')
ax.set_ylabel('Eje 2')
ax.set_zlabel('Eje 3')
pyplot.show()